In [445]:
import numpy as np
import matplotlib as mp
import time
import random

In [464]:
# parameters for genetic algorithm
max_weight = 10
population_size = 15
mutation_probability = 0.08
crossover_probability = 0.8
generations = 40
stagnation_limit=10
e_threshold = 0.01
elitism_size = 1

In [447]:
# loading all the points from the data files into arrays

tset_file = open("data/training_set_v2", "r")
tlabel_file = open("data/training_labels_v2", "r")
vset_file = open("data/validation_set_v2", "r")
vlabel_file = open("data/validation_labels_v2", "r")

tset = []
tlabel = []
vset = []
vlabel = []

for i in tset_file.readlines():
    x0,x1,x2 = i.split(sep=',')
    tset += [[float(x0),float(x1),float(x2)]]

tset_file.close()


for i in tlabel_file.readlines():
    di = i.split()
    tlabel += [float(di[0])]

tlabel_file.close()

for i in vset_file.readlines():
    x0,x1,x2 = i.split(sep=',')
    vset += [[float(x0),float(x1),float(x2)]]

vset_file.close()


for i in vlabel_file.readlines():
    di = i.split()
    vlabel += [float(di[0])]

vlabel_file.close()

In [448]:
#ao ∈ [0, 2], a1 ∈ [−2, 0], and a2 ∈ [−1, 1].
def my_fun(x,y,a0,a1,a2):
    return (a0*np.cbrt(x-5)) + a1*np.cbrt(y+5) + a2

def obj_function(pred, actVal):
    #computing the mean square error
    return np.square(np.subtract(pred, actVal)).mean() 

def gen_population(size):
    population = []
    for _ in range(size):
        population.append([random.randrange(0,3), random.randrange(-2,1), random.randrange(-1,2)])
    
    print("Generated population of size ", size)
    return population

def evaluate(chromosome):
    predictions = []
    
    for i in range(len(tlabel)):
        predictions.append(my_fun(tset[i][0],tset[i][0],chromosome[0], chromosome[1], chromosome[2]))
    
    fitness = obj_function(predictions, tlabel)
    return -fitness


In [449]:
def roulette_wheel_selection(population):
    fitness_values = [evaluate(chromosome) for chromosome in population]
    total_fitness = sum(fitness_values)

    # Calculate selection probabilities
    selection_probs = [fitness / total_fitness for fitness in fitness_values]

    # Perform roulette wheel spin
    selected_index = -1
    spin = random.uniform(0, 1)
    cumulative_prob = 0.0

    for i, prob in enumerate(selection_probs):
        cumulative_prob += prob
        if spin <= cumulative_prob:
            selected_index = i
            break

    return population[selected_index]
def select_parents(population):
    parents = []
    #generate parent pairs array the half the size of the population
    for _ in range(len(population)//2):
        parents.append([roulette_wheel_selection(population),roulette_wheel_selection(population)])
    return parents


In [450]:

# Function to perform uniform crossover between two parents
def crossover(parent1, parent2):
    
    # Create an empty child chromosome
    child1 = [None]*3
    child2 = [None]*3
    crossover = False
    # Uniformly select genes from parents to create the child chromosome
    for _ in range(len(parent1)):
        if random.uniform(0,1) <= crossover_probability:
            r = random.uniform(0,1)
            child1 = parent1 * r + parent2*(1-r)
            child2 = parent1 * (1-r) + parent2 * r
            crossover = True
    if crossover:
        print(f"Crossover beetween {parent1} and {parent2} Resulted in {child1} and {child2}")
        return child1, child2
    else:
        print(f"No Crossover between {parent1} and {parent2}")
        return parent1, parent2
    

In [451]:
def mutate(chromosome):
    mutated = False
    for i in range(len(chromosome)):
        if random.uniform(0,1) <= mutation_probability:
            print(f"Performing mutation on chromosome {chromosome} at index {i}")
            mutated = True
            if i == 0:
                chromosome[i] = random.uniform(0, 3)
            if i == 1:
                chromosome[i] = random.uniform(-2, 1)
            if i == 2:
                chromosome[i] = random.uniform(-1, 2)
            
            
    if mutated:
        print(f"Resulting chromosome: {chromosome}\n")
    else:
        print(f"No mutation for {chromosome}\n")
    
    return chromosome

    
    

In [452]:
def get_best(population):
    fitness_values = []
    elites = []
    for chromosome in population:
        fitness_values.append(evaluate(chromosome))
    elite_indices = sorted(range(len(fitness_values)), key=lambda i: fitness_values[i])[:elitism_size]
    elites = [population[i] for i in elite_indices]
    
    return elites[0], elites



In [466]:
def genetic_algorithm_e():
    # Initialization
    pop = gen_population(population_size)
    best_fitness = float('inf') 
    stagnation_count = 0
    generation = 0
    best_candidate = []
    while(True):
        generation +=1
        parent_pairs = select_parents(pop)
        current_best_chrom, elites = get_best(pop)
        current_best_fitness = evaluate(current_best_chrom)

        diversity_check = abs((current_best_fitness-best_fitness)/current_best_fitness)
        # Check for stagnation in the best fitness
        
        if diversity_check < e_threshold:
            stagnation_count += 1
        else:
            stagnation_count = 0
        
        best_fitness = current_best_fitness

        if stagnation_count >= stagnation_limit:
            print(f"Terminating due to no change in best fitness for {stagnation_limit} generations.")
            break
        
        # Mutation and selection
        new_population = []
        for i in parent_pairs:
            child1, child2 = crossover(np.array(i[0]),np.array(i[1]))
            child1 = mutate(child1)
            child2 = mutate(child2)
            new_population.append(child1)
            new_population.append(child2)

        # Update population for the next generation
        pop = elites + new_population
        
        # Optional: Display current generation's best individual and its fitness
        best_candidate, _ = get_best(pop)
    print(f"Generation {generation}: Best Fitness = {best_fitness}, Best Candidate = {best_candidate}")

   

genetic_algorithm_e()

Generated population of size  15
Crossover beetween [ 0 -2  0] and [ 0 -2  0] Resulted in [ 0. -2.  0.] and [ 0. -2.  0.]
No mutation for [ 0. -2.  0.]

No mutation for [ 0. -2.  0.]

Crossover beetween [ 2 -2 -1] and [ 2 -2 -1] Resulted in [ 2. -2. -1.] and [ 2. -2. -1.]
No mutation for [ 2. -2. -1.]

Performing mutation on chromosome [ 2. -2. -1.] at index 1
Resulting chromosome: [ 2.        -0.7838153 -1.       ]

Crossover beetween [ 1 -1  0] and [ 2 -1  1] Resulted in [ 1.90946261 -1.          0.90946261] and [ 1.09053739 -1.          0.09053739]
No mutation for [ 1.90946261 -1.          0.90946261]

Performing mutation on chromosome [ 1.09053739 -1.          0.09053739] at index 2
Resulting chromosome: [ 1.09053739 -1.          1.62974766]

Crossover beetween [ 2 -2  0] and [2 0 0] Resulted in [ 2.         -1.59563706  0.        ] and [ 2.         -0.40436294  0.        ]
No mutation for [ 2.         -1.59563706  0.        ]

No mutation for [ 2.         -0.40436294  0.        ]


In [454]:
def genetic_algorithm():
    # Initialization
    pop = gen_population(population_size)
    best_fitness = float('inf') 
    stagnation_count = 0

    for generation in range(generations):
        
        parent_pairs = select_parents(pop)
        current_best_fitness = evaluate(get_best(pop))

        # Check for stagnation in the best fitness
        if current_best_fitness == best_fitness:
            stagnation_count += 1
        else:
            stagnation_count = 0
            best_fitness = current_best_fitness

        if stagnation_count >= stagnation_limit:
            print(f"Terminating due to no change in best fitness for {stagnation_limit} generations.")
            break

        # Mutation and selection
        new_population = []
        for i in parent_pairs:
            child1, child2 = crossover(np.array(i[0]),np.array(i[1]))
            child1 = mutate(child1)
            child2 = mutate(child2)
            new_population.append(child1)
            new_population.append(child2)

        # Update population for the next generation
        pop = new_population
        
        # Optional: Display current generation's best individual and its fitness
        best_candidate = get_best(pop)
        print(f"{len(pop)} Generation {generation + 1}: Best Fitness = {best_fitness}, Best Candidate = {best_candidate}")

    return pop

genetic_algorithm()

Generated population of size  15


IndexError: tuple index out of range